In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import os
import sys
from tqdm.notebook import tqdm # Используем версию для ноутбуков
from sklearn.metrics import confusion_matrix
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

from config import config as main_config 
from utils.metrics import calculate_levenshtein_mean, calculate_cer
from utils.decoding import decode_predictions, decode_targets
from data.dataset import MorseDataset 
from data.collate import collate_fn 
from models.crnn import CRNNModel_4Layer

log_file_path = main_config.LOG_FILE
history_df = None

try:
    history_df = pd.read_csv(log_file_path)
    print(f"Лог файл загружен: {log_file_path}")
    display(history_df.head())

    # Строим графики
    epochs_range = history_df['Epoch']

    plt.figure(figsize=(18, 5))

    plt.subplot(1, 4, 1)
    plt.plot(epochs_range, history_df['Train Loss'], label='Train Loss')
    plt.plot(epochs_range, history_df['Val Loss'], label='Validation Loss')
    plt.legend(loc='best'); plt.title('Loss'); plt.xlabel('Epoch'); plt.grid(True)

    plt.subplot(1, 4, 2)
    plt.plot(epochs_range, history_df['Val Levenshtein'], label='Validation Levenshtein')
    plt.legend(loc='best'); plt.title('Levenshtein Distance'); plt.xlabel('Epoch'); plt.grid(True)
    best_lev_epoch = history_df.loc[history_df['Val Levenshtein'].idxmin()]
    plt.scatter(best_lev_epoch['Epoch'], best_lev_epoch['Val Levenshtein'], color='red', s=50, label=f'Best: {best_lev_epoch["Val Levenshtein"]:.4f} (Ep {int(best_lev_epoch["Epoch"])})')
    plt.legend(loc='best')


    plt.subplot(1, 4, 3)
    if 'Val CER' in history_df.columns and not history_df['Val CER'].isnull().all():
         plt.plot(epochs_range, history_df['Val CER'], label='Validation CER')
         plt.legend(loc='best'); plt.title('Character Error Rate (CER)'); plt.xlabel('Epoch'); plt.grid(True)
    else:
         plt.text(0.5, 0.5, 'CER data not available', horizontalalignment='center', verticalalignment='center', transform=plt.gca().transAxes)
         plt.title('Character Error Rate (CER)')

    plt.subplot(1, 4, 4)
    if 'LR' in history_df.columns:
        plt.plot(epochs_range, history_df['LR'], label='Learning Rate')
        plt.legend(loc='best'); plt.title('Learning Rate'); plt.xlabel('Epoch'); plt.grid(True); plt.yscale('log') 
    else:
         plt.text(0.5, 0.5, 'LR data not available', horizontalalignment='center', verticalalignment='center', transform=plt.gca().transAxes)
         plt.title('Learning Rate')


    plt.suptitle(f'Результаты обучения модели: {main_config.MODEL_NAME}', fontsize=16)
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

except FileNotFoundError:
    print(f"Ошибка: Лог файл не найден по пути {log_file_path}")
except Exception as e:
    print(f"Ошибка при загрузке или отрисовке логов: {e}")

DEVICE = main_config.DEVICE
best_model = None
best_checkpoint_path = None
loaded_config = None
char_to_int = None
int_to_char = None
blank_index = -1

try:
    best_checkpoint_path = os.path.join(main_config.CHECKPOINT_DIR, f"{main_config.MODEL_NAME}_best.pth")
    if not os.path.exists(best_checkpoint_path):
         available_checkpoints = [f for f in os.listdir(main_config.CHECKPOINT_DIR) if f.endswith(".pth") and main_config.MODEL_NAME in f]
         if not available_checkpoints:
             raise FileNotFoundError(f"Чекпоинты не найдены в {main_config.CHECKPOINT_DIR}")
         best_checkpoint_path = os.path.join(main_config.CHECKPOINT_DIR, available_checkpoints[0])
         print(f"Файл best.pth не найден, используется: {os.path.basename(best_checkpoint_path)}")

    print(f"\nЗагрузка лучшей модели для анализа: {os.path.basename(best_checkpoint_path)}")
    checkpoint = torch.load(best_checkpoint_path, map_location=DEVICE)

    if 'config' not in checkpoint or 'char_to_int' not in checkpoint or 'int_to_char' not in checkpoint or 'blank_index' not in checkpoint:
        raise ValueError("Необходимые данные (config, char_map, blank_index) не найдены в чекпоинте!")

    loaded_config = checkpoint['config']
    char_to_int = checkpoint['char_to_int']
    int_to_char = checkpoint['int_to_char']
    blank_index = checkpoint['blank_index']
    NUM_CLASSES = len(char_to_int)

    print("Конфигурация модели из чекпоинта:")
    print(f"  Тип: {loaded_config.get('MODEL', {}).get('type', 'N/A')}")
    print(f"  Размер RNN: {loaded_config.get('MODEL', {}).get('rnn_hidden_size', 'N/A')}")
    print(f"  Слои RNN: {loaded_config.get('MODEL', {}).get('rnn_num_layers', 'N/A')}")
    print(f"  n_mels: {loaded_config.get('AUDIO', {}).get('n_mels', 'N/A')}")

    model_cfg = loaded_config['MODEL']
    audio_cfg = loaded_config['AUDIO']

    if model_cfg['type'] == 'CRNNModel_4Layer':
        best_model = CRNNModel_4Layer(
            n_features=audio_cfg['n_mels'], num_classes=NUM_CLASSES,
            rnn_hidden_size=model_cfg['rnn_hidden_size'], num_rnn_layers=model_cfg['rnn_num_layers'],
            cnn_dropout=0.0, rnn_dropout=0.0 
        ).to(DEVICE)
    else:
        raise ValueError(f"Неизвестный тип модели '{model_cfg['type']}' в чекпоинте")

    best_model.load_state_dict(checkpoint['model_state_dict'])
    best_model.eval() 
    print(f"Лучшая модель (Эпоха {checkpoint.get('epoch', 'N/A')}, Metric: {checkpoint.get('val_metric', 'N/A'):.4f}) загружена.")

except FileNotFoundError:
    print(f"Ошибка: Чекпоинт не найден в '{main_config.CHECKPOINT_DIR}'")
    best_model = None 
except Exception as e:
    print(f"Ошибка загрузки лучшей модели: {e}")
    traceback.print_exc()
    best_model = None

val_loader_final = None
if best_model and loaded_config and char_to_int:
    print("\nПодготовка валидационного лоадера для финальной оценки...")
    try:
        train_df_full = pd.read_csv(main_config.TRAIN_CSV_PATH)
        train_df_full[main_config.TARGET_COLUMN] = train_df_full[main_config.TARGET_COLUMN].fillna('').astype(str)
        train_df_filtered = train_df_full[train_df_full[main_config.TARGET_COLUMN].str.len() > 0].copy()

        # Используем тот же random_state для split, чтобы получить ту же val выборку
        _, val_df_final = train_test_split(
            train_df_filtered,
            test_size=main_config.TRAINING['validation_split_size'], # Используем размер сплита из основного конфига
            random_state=main_config.SEED
        )
        val_df_final = val_df_final.reset_index(drop=True)

        # Создаем датасет с параметрами из ЗАГРУЖЕННОГО конфига
        val_dataset_final = MorseDataset(
            data_frame=val_df_final, audio_base_path=main_config.AUDIO_BASE_PATH,
            char_map=char_to_int, # Используем словарь из чекпоинта
            file_path_column=loaded_config['FILE_PATH_COLUMN'],
            target_column=loaded_config['TARGET_COLUMN'],
            test_id_column=loaded_config['TEST_ID_COLUMN'],
            audio_cfg=loaded_config["AUDIO"],
            preproc_cfg=loaded_config["PREPROCESSING"],
            aug_cfg=None,
            is_train=False
        )

        val_loader_final = DataLoader(
            val_dataset_final, batch_size=main_config.INFERENCE['batch_size'], 
            shuffle=False, collate_fn=collate_fn, num_workers=0
        )
        print(f"Валидационный лоадер создан ({len(val_dataset_final)} сэмплов).")

    except Exception as e:
        print(f"Ошибка создания валидационного лоадера: {e}")
        val_loader_final = None


if best_model and val_loader_final and int_to_char and blank_index != -1:
    print("\nРасчет финальных метрик и Confusion Matrix на валидационной выборке (Greedy Decode)...")
    all_val_preds_final, all_val_targets_final = [], []
    val_iter_final = tqdm(val_loader_final, desc="Final Validation")

    with torch.no_grad():
        for batch in val_iter_final:
            try:
                if not isinstance(batch, (tuple, list)) or len(batch) != 4: continue
                spec_batch, spec_len_batch, target_batch, target_len_batch = batch
                if spec_batch.numel() == 0: continue

                spec_batch = spec_batch.to(DEVICE)
                spec_len_batch = spec_len_batch.to(DEVICE)
                target_batch = target_batch.to(DEVICE)
                target_len_batch = target_len_batch.to(DEVICE)


                log_probs, output_lengths = best_model(spec_batch, spec_len_batch)
                output_lengths = torch.clamp(output_lengths, max=log_probs.shape[0])

                batch_preds = decode_predictions(log_probs, int_to_char, blank_index)
                batch_targets = decode_targets(target_batch.cpu(), target_len_batch.cpu(), int_to_char) 
                all_val_preds_final.extend(batch_preds)
                all_val_targets_final.extend(batch_targets)

            except Exception as e:
                print(f"\nОшибка на шаге финальной валидации: {e}")
                continue

    if all_val_targets_final:
        final_lev = calculate_levenshtein_mean(all_val_preds_final, all_val_targets_final)
        final_cer = calculate_cer(all_val_preds_final, all_val_targets_final)
        print(f"\nФинальные метрики на Validation Set (Best Model, Greedy Decode):")
        print(f"  - Levenshtein Distance: {final_lev:.4f}")
        print(f"  - Character Error Rate (CER): {final_cer:.4f}")

        try:
            print("\nПостроение Confusion Matrix...")
            pred_chars = "".join(all_val_preds_final)
            target_chars = "".join(all_val_targets_final)

            max_len_cm = 100000
            if len(target_chars) > max_len_cm:
                 print(f"(Слишком много символов ({len(target_chars)}), матрица строится на {max_len_cm} случайных)")
                 indices = np.random.choice(len(target_chars), max_len_cm, replace=False)
                 pred_chars_list = list(pred_chars)
                 target_chars_list = list(target_chars)
                 pred_chars = "".join([pred_chars_list[i] for i in indices])
                 target_chars = "".join([target_chars_list[i] for i in indices])


            labels = sorted(list(set(target_chars) | set(pred_chars)))
            if not labels:
                 print("Нет символов для построения матрицы ошибок.")
            else:
                print(f"Уникальные символы для матрицы ({len(labels)}): {''.join(labels)}")
                cm = confusion_matrix(list(target_chars), list(pred_chars), labels=labels)
                cm_df = pd.DataFrame(cm, index=labels, columns=labels)

                plt.figure(figsize=(max(10, len(labels)*0.4), max(8, len(labels)*0.35))) 
                sns.heatmap(cm_df, annot=True, fmt='d', cmap='Blues')
                plt.title(f'Confusion Matrix - Validation Set ({main_config.MODEL_NAME})')
                plt.ylabel('Actual Characters')
                plt.xlabel('Predicted Characters')
                plt.show()

                errors = []
                total_errors = 0
                for i, label_true in enumerate(labels):
                    for j, label_pred in enumerate(labels):
                        if i != j and cm[i, j] > 0:
                            errors.append(((label_true, label_pred), cm[i, j]))
                            total_errors += cm[i,j]

                if errors:
                    errors.sort(key=lambda item: item[1], reverse=True)
                    print(f"\nНаиболее частые ошибки ({total_errors} всего ошибок):")
                    for pair, count in errors[:20]: # Показать топ N
                        true_char = pair[0] if pair[0] != ' ' else "' '"
                        pred_char = pair[1] if pair[1] != ' ' else "' '"
                        print(f"  {true_char} -> {pred_char}: {count}")
                else:
                     print("\nОшибок не найдено!")

        except Exception as e:
            print(f"Ошибка построения Confusion Matrix: {e}")
            traceback.print_exc()
    else:
         print("Не удалось получить предсказания/таргеты для финального анализа.")
elif not best_model:
    print("Анализ невозможен, так как лучшая модель не была загружена.")
else:
    print("Анализ невозможен, так как не удалось создать валидационный лоадер или отсутствуют словари.")

# +
# Опционально: Показать примеры предсказаний
if 'all_val_preds_final' in locals() and 'all_val_targets_final' in locals() and all_val_targets_final:
    print("\nПримеры предсказаний на валидационной выборке:")
    num_samples_to_show = 15
    indices_to_show = np.random.choice(len(all_val_targets_final), min(num_samples_to_show, len(all_val_targets_final)), replace=False)

    print("-" * 50)
    for i in indices_to_show:
        target = all_val_targets_final[i]
        pred = all_val_preds_final[i]
        is_correct = "(Correct)" if target == pred else "(Incorrect)"
        print(f"Target:    {target}")
        print(f"Predicted: {pred} {is_correct}")
        print("-" * 50)
# -